In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table
import cmasher as cmr
import CFigTools.CustomFigure as CF
import gpscssmodels
import pandas as pd
import plotting_functions

In [2]:
# Paths and non-changing variables 
master_data_dir = "/data/raw_data/"
data_dir = "/data/LBA/catalogues/"
frequencies = ["2GHz", "8GHz"]
lba_targets = ["j0227-0621", "j0322-482", "j2239-451"]
mwa_targets = ["GLEAM J022744-062106", "GLEAM J032237-482010", "GLEAM J223933-451414"]



In [3]:
# Setting arrays for masks and extracting sources
exts = [
    "107",
    "115",
    "123",
    "130",
    "143",
    "150",
    "158",
    "166",
    "174",
    "181",
    "189",
    "197",
    "204",
    "212",
    "220",
    "227",
]
mwa_mask_names = ["Name", "S_076", "S_084", "S_092", "S_099","S_076_err", "S_084_err", "S_092_err", "S_099_err"]
mwa_2013_fluxes = ["S_076", "S_084", "S_092", "S_099"]
mwa_2014_fluxes = []
mwa_2013_errors = ["S_076_err", "S_084_err", "S_092_err", "S_099_err"]
mwa_2014_errors = []
for ext in exts:
    mwa_mask_names.extend((f"S_{ext}_yr1", f"S_{ext}_yr2",f"local_rms_{ext}_yr1", f"local_rms_{ext}_yr2"))
    mwa_2013_fluxes.append(f"S_{ext}_yr1")
    mwa_2014_fluxes.append(f"S_{ext}_yr2")
    mwa_2013_errors.append(f"local_rms_{ext}_yr1")
    mwa_2014_errors.append(f"local_rms_{ext}_yr2")
# Extras
xtra_fluxes = [
    "S_tgss",
    "S_mrc",
    "S_sumss",
    "S_nvss",
    "S_vlssr",
    "S20",
    "S8",
    "S5",
    "total_flux_source",  # RACS
]

In [4]:
master_pop_pd = pd.read_csv(f"{master_data_dir}/master_pop_extended.csv")
# master_pop_pd = Table.read(f"{master_data_dir}/master_pop_extended.fits")
lba_mask = master_pop_pd["Name"].isin(mwa_targets)
lba_master_pop = master_pop_pd[lba_mask]
lbapop_extra_fluxes = lba_master_pop[xtra_fluxes]
lba_pop = lba_master_pop[mwa_mask_names]
print(np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][mwa_2013_fluxes].to_numpy()))
print(np.squeeze(lba_pop.loc[lba_pop["Name"]==mwa_targets[0]].to_numpy()))
# print(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]])

[0.28222173 0.35457686 0.30922073 0.28430387 0.41807082 0.36974546
 0.41494986 0.4548864  0.46420056 0.47677046 0.50158477 0.50865954
 0.5301754  0.54209995 0.5545837  0.5552021  0.5342091  0.5670117
 0.5414796  0.56120336]
['GLEAM J022744-062106' 0.28222173 0.35457686 0.30922073 0.28430387
 0.08509528 0.06430224 0.05570395 0.05481943 0.41807082 0.2810934
 0.038487468 0.031815585 0.36974546 0.28716403 0.03184621 0.025730874
 0.41494986 0.34148353 0.0305772 0.022218546 0.4548864 0.35462028
 0.028244225 0.021078106 0.46420056 0.36847794 0.017901305 0.014526561
 0.47677046 0.40623057 0.016408706 0.012181905 0.50158477 0.4077316
 0.015134427 0.010965364 0.50865954 0.4111097 0.013937391 0.01024465
 0.5301754 0.42966878 0.015192327 0.010483272 0.54209995 0.4484246
 0.014547369 0.009412393 0.5545837 0.44635257 0.013424073 0.008574207
 0.5552021 0.44736254 0.013163328 0.008555906 0.5342091 0.4576202
 0.015675183 0.0097518135 0.5670117 0.46242118 0.01567657 0.009869933
 0.5414796 0.46472543 0.0

In [5]:
src_13_fluxes = np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][mwa_2013_fluxes].to_numpy())
src_14_fluxes = np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][mwa_2014_fluxes].to_numpy())
src_14_fluxes = np.insert(src_14_fluxes, 0, [np.nan, np.nan, np.nan, np.nan])
src_13_errs = np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][mwa_2013_errors].to_numpy())
src_14_errs = np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][mwa_2014_errors].to_numpy())
src_14_errs = np.insert(src_14_errs, 0, [np.nan, np.nan, np.nan, np.nan])
extra_fluxes = np.squeeze(lba_master_pop.loc[lba_master_pop["Name"]==mwa_targets[0]][xtra_fluxes].to_numpy())
print(np.shape(extra_fluxes))
extra_fluxes[0] = extra_fluxes[0]*0.001
extra_fluxes[2] = extra_fluxes[2]*0.001
extra_fluxes[3] = extra_fluxes[3]*0.001
extra_fluxes[5] = extra_fluxes[5]*0.001
extra_fluxes[6] = extra_fluxes[6]*0.001
extra_fluxes[7] = extra_fluxes[7]*0.001
extra_fluxes[8] = extra_fluxes[8]*0.001
    

(9,)


In [6]:
# Will need to be made general

lba_2ghz_flux = []
local_rms = []
for tar in lba_targets:
    try:
        t = Table.read(f"{data_dir}{tar}_2GHz.fits")
        print(t)
        lba_2ghz_flux.append(np.squeeze(np.array(t["int_flux"])))
        local_rms.append(np.array(t["local_rms"]))
        
    except FileNotFoundError:
        print(f"No catalogue for {tar} 2GHz")

print(lba_2ghz_flux)

lba_8ghz_flux = []
local_rms = []
for tar in lba_targets:
    try:
        t = Table.read(f"{data_dir}{tar}_8GHz.fits")
        print(t)
        lba_8ghz_flux.append(np.squeeze(np.array(t["int_flux"])))
        local_rms.append(np.array(t["local_rms"]))
        
    except FileNotFoundError:
        print(f"No catalogue for {tar} 8GHz")

print(lba_8ghz_flux)

island source   background   local_rms   ...    psf_a       psf_b       psf_pa 
------ ------ ------------- ------------ ... ----------- ------------ ---------
     1      0 0.00057751953 0.0035204089 ... 0.009568015 0.0031954367  6.955581
     2      0   0.000444626 0.0033355528 ... 0.009568015 0.0031954367 6.9555817
     3      0 0.00046352297 0.0034423615 ... 0.009568015 0.0031954367 6.9555826
No catalogue for j0322-482 2GHz
No catalogue for j2239-451 2GHz
[array([0.07550298, 0.21996112, 0.10833634], dtype=float32)]
island source   background   local_rms   ...    psf_a        psf_b      psf_pa 
------ ------ ------------- ------------ ... ------------ ----------- ---------
     1      0   2.41316e-05 0.0012398985 ... 0.0038990302 0.001737589 8.1431875
     2      0  9.810134e-06 0.0013578946 ... 0.0038990302 0.001737589  8.143188
     3      0 2.6209766e-05 0.0012140629 ... 0.0038990302 0.001737589  8.143189
No catalogue for j0322-482 8GHz
No catalogue for j2239-451 8GHz
[array([0.0

In [7]:
import plotting_functions
src_fluxes = [src_13_fluxes, src_14_fluxes]
err_src_fluxes = [src_13_errs, src_14_errs]
lba_2ghz_flux = lba_2ghz_flux[0]
lba_8ghz_flux = lba_8ghz_flux[0]
for i in range(len(lba_2ghz_flux)):
    print(lba_2ghz_flux[i])
plotting_functions.plt_sed("/data/LBA/plots/", mwa_targets[0], src_fluxes, err_src_fluxes, extra_fluxes,lba_2ghz_flux, lba_8ghz_flux, frequency=np.array(
        [
            0.076,
            0.084,
            0.092,
            0.099,
            0.107,
            0.115,
            0.122,
            0.130,
            0.143,
            0.151,
            0.158,
            0.166,
            0.174,
            0.181,
            0.189,
            0.197,
            0.204,
            0.212,
            0.220,
            0.227]))

0.075502984
0.21996112
0.108336344


<Figure size 432x288 with 0 Axes>